In [ ]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")

In [ ]:
guideModules = as.data.frame(read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv",
                                 stringsAsFactors = F, strip.white = T, header = TRUE, row.names=1))
geneModules = as.data.frame(read.csv("./../TextFiles/ME_GeneModules_leiden_11_Modules.csv",
                                 stringsAsFactors = F, strip.white = T, header = TRUE, row.names=1))
geneModules$GeneName = rownames(geneModules)
coefsSgn = as.data.frame(read.csv("./../MixedEffectLMOutputs/ME_SignificantBetaCoefs.csv",
                                  header=TRUE, row.names=1))

In [ ]:
rownames(coefsSgn) <- sapply(rownames(coefsSgn), function(x){k=strsplit(x,"_")
                                      return(k[[1]][2])})

In [ ]:
allTestRes <- list()

for (guideGroup in sort(unique(guideModules$GuideGroup))){
    for (geneGroup in sort(unique(geneModules$GeneGroup))){
        
        blockBetas <- coefsSgn[guideModules[guideModules$GuideGroup == guideGroup, "GuideName"], 
                               geneModules[geneModules$GeneGroup == geneGroup, "GeneName"]]
        blockBetas <- melt(blockBetas)$value
        allSampledBetas <- sample(melt(coefsSgn)$value,100000)
        meanDif <- mean(blockBetas) - mean(allSampledBetas)
        
        k <- wilcox.test(blockBetas, allSampledBetas, alternative = c("greater"))
        allTestRes <- lappend(allTestRes, c(guideGroup, geneGroup, "greater", k$p.value, meanDif))
        
        k <- wilcox.test(blockBetas, allSampledBetas, alternative = c("less"))
        allTestRes <- lappend(allTestRes, c(guideGroup, geneGroup, "less", k$p.value, meanDif))
    }
}

In [ ]:
allTestRes <- data.frame(do.call(rbind, allTestRes), stringsAsFactors = FALSE)
colnames(allTestRes) <- c("guideGroup", "geneGroup", "lessOrGreater", "P_value", "MeanDiff")
allTestRes$P_value <- as.numeric(allTestRes$P_value)
allTestRes$FDR <- p.adjust(allTestRes$P_value)
allTestRes <- allTestRes[allTestRes$FDR < 0.05,]
allTestRes$guideGroup = paste0("K",allTestRes$guideGroup)

In [ ]:
write.csv(allTestRes,"./../TextFiles/GuideModuleGeneModuleEffectSignificance.csv" , row.names=FALSE)